In [1]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
os.environ['_JAVA_OPTIONS'] = f'-Djava.io.tmpdir={localfs_path}'

import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '10G'}, # I don't know what should be here
    default_reference='GRCh38'
) 

Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4204276/
Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4204276/


23/07/30 13:35:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://ac0067:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.113-cf32652c5077
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230730-1335-0.2.113-cf32652c5077.log


In [3]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [4]:
def run_pca_no_filter(mtx, mtx_path, suffix):
    
    for_pca = mtx.sample_rows(0.1)

    for_pca = for_pca.checkpoint(localfs_path+'subset_'+suffix+mtx_path)
    for_pca = hl.read_matrix_table(localfs_path+'subset_'+suffix+mtx_path)
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT, k=20)
    
    mtx = mtx.annotate_cols(scores_no_filter = pcs[mtx.s].scores)
    mtx = mtx.checkpoint(localfs_path+'after_pca_no_filters_'+suffix+mtx_path)
    
    return(mtx)

### read in the polish without zeros subset the controls randomly allocated (to have 100 control samples), the matrix table already has outliers removed

In [7]:
pw_vs_gts = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/pw_vs_gts_pca.mt'
)

In [8]:
pw_vs_gts.count()

(3035372, 1240)

In [9]:
samples_to_exclude = ['S_7288', 'S_7289', 'S_7290', 'WGS_6827', 'WGS_6835', 'WGS_5', 'WGS_8']
#these are low quality samples to exclude

pw_vs_gts = pw_vs_gts.filter_cols(
    hl.literal(samples_to_exclude).contains(pw_vs_gts.s),
    keep = False
)

#exclude duplicated sample 
pw_vs_gts = pw_vs_gts.distinct_by_col()

In [10]:
pw_vs_gts.count()

(3035372, 1237)

In [17]:
# I am going to divide the controls into 10 ~equal groups:
pw_vs_gts = pw_vs_gts.annotate_cols(
    subset_no = hl.if_else(
        pw_vs_gts.group == 'GTS',
        0,
        hl.rand_int32(1, 11, seed=0)
    )
)

In [18]:
pw_vs_gts.aggregate_cols(hl.agg.counter(pw_vs_gts.subset_no))

{0: 37,
 1: 111,
 2: 126,
 3: 134,
 4: 104,
 5: 112,
 6: 110,
 7: 120,
 8: 134,
 9: 120,
 10: 129}

In [26]:
mts = []

for i in list(range(1, 11)):
    
    to_out = pw_vs_gts.filter_cols(pw_vs_gts.subset_no == i)
    to_out = to_out.s.take(to_out.count()[1]-100)
    
    mt = pw_vs_gts.filter_cols(
        (pw_vs_gts.subset_no == 0) | (pw_vs_gts.subset_no == i)
    )
    
    mt = mt.filter_cols(
        hl.literal(to_out).contains(mt.s),
        keep = False
    )
    
    print(mt.count())
        
    mts.append(mt)

(3035372, 137)
(3035372, 137)
(3035372, 137)
(3035372, 137)
(3035372, 137)
(3035372, 137)
(3035372, 137)
(3035372, 137)
(3035372, 137)
(3035372, 137)


In [27]:
mts_paths = []

for i in list(range(1,11)):
    mts_paths.append(str(i)+'.mt')

In [ ]:
for idx, mt in enumerate(mts):
    mts[idx] = run_pca_no_filter(mt, mts_paths[idx], 'gts_vs_pol_subseted')

2023-07-30 13:52:43.967 Hail: INFO: wrote matrix table with 303055 rows and 137 columns in 488 partitions to /localfs/4204276/subset_gts_vs_pol_subseted1.mt
    Total size: 2.25 GiB
    * Rows/entries: 2.25 GiB
    * Columns: 44.65 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  1685 rows (12.69 MiB)
2023-07-30 13:53:28.564 Hail: INFO: hwe_normalize: found 279006 variants after filtering out monomorphic sites.
2023-07-30 13:53:33.027 Hail: INFO: pca: running PCA with 20 components.../ 488]
2023-07-30 13:54:28.699 Hail: INFO: wrote table with 0 rows in 0 partitions to /localfs/4204276/tmp_hail/persist_tableqhdTIZU52b
    Total size: 22.61 KiB
    * Rows: 0.00 B
    * Globals: 22.61 KiB
    * Smallest partition: N/A
    * Largest partition:  N/A
2023-07-30 13:55:50.794 Hail: INFO: wrote matrix table with 3035372 rows and 137 columns in 488 partitions to /localfs/4204276/after_pca_no_filters_gts_vs_pol_subseted1.mt
    Total size: 22.27 GiB
